In [19]:
ticker = 'lren3'

In [20]:
import pandas as pd
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [26]:
url = f'https://opcoes.net.br/listaopcoes/completa?au=False&uinhc=0&idLista=ML&idAcao={ticker}&listarVencimentos=true&cotacoes=true'
response = requests.get(url, verify=False).json()

# proximas datas de vencimento
datas = [[i['value'], i['text']] for i in response['data']['vencimentos']]
# datas

In [22]:
derivativos = []

for vcto in datas:
    url = f'https://opcoes.net.br/listaopcoes/completa?au=False&uinhc=0&idLista=ML&idAcao={ticker}&listarVencimentos=false&cotacoes=true&vencimentos={vcto[0]}'
    response = requests.get(url, verify=False).json()

    # CORREÇÃO: Use vcto[0] em vez de vcto[1]
    vcto_date = vcto[0]  # '2026-01-16' em vez de '16/01'
    derivativos.extend(
        [ticker, i[0].split('_', 1)[0], i[2], i[3], vcto_date, i[5], i[8]] 
        for i in response['data']['cotacoesOpcoes']
    )

# CORREÇÃO: DataFrame FORA do loop
colunas = ['ATIVO_OBJ', 'ATIVO', 'TIPO', 'MOD', 'DT_VCTO', 'STRIKE', 'PRECO']
opcoes = pd.DataFrame(derivativos, columns=colunas)

# Agora funciona porque vcto_date tem ano completo
opcoes['DT_VCTO'] = pd.to_datetime(opcoes['DT_VCTO'])

In [23]:
opcoes

,ATIVO_OBJ,ATIVO,TIPO,MOD,DT_VCTO,STRIKE,PRECO
0,lren3,LRENA1,CALL,A,2026-01-16,16.96,0.01
1,lren3,LRENA101,CALL,A,2026-01-16,9.96,NaN
2,lren3,LRENA106,CALL,A,2026-01-16,10.46,NaN
3,lren3,LRENA109,CALL,E,2026-01-16,10.71,NaN
4,lren3,LRENA111,CALL,A,2026-01-16,10.96,NaN
...,...,...,...,...,...,...,...
595,lren3,LRENL26,CALL,A,2027-12-17,25.81,0.90
596,lren3,LRENX200,PUT,E,2027-12-17,19.81,4.49
597,lren3,LRENX22,PUT,E,2027-12-17,21.81,NaN
598,lren3,LRENX24,PUT,E,2027-12-17,23.81,NaN


In [24]:
from IPython.display import  display, HTML

display(HTML(opcoes[opcoes['TIPO'] == 'PUT'].dropna().to_html()))

,ATIVO_OBJ,ATIVO,TIPO,MOD,DT_VCTO,STRIKE,PRECO
56,lren3,LRENM1,PUT,E,2026-01-16,16.96,3.60
57,lren3,LRENM101,PUT,E,2026-01-16,9.96,0.02
58,lren3,LRENM106,PUT,E,2026-01-16,10.46,0.02
59,lren3,LRENM109,PUT,E,2026-01-16,10.71,0.01
60,lren3,LRENM111,PUT,E,2026-01-16,10.96,0.01
61,lren3,LRENM116,PUT,E,2026-01-16,11.46,0.01
62,lren3,LRENM119,PUT,E,2026-01-16,11.71,0.01
63,lren3,LRENM121,PUT,E,2026-01-16,11.96,0.02
64,lren3,LRENM124,PUT,E,2026-01-16,12.21,0.02
65,lren3,LRENM126,PUT,E,2026-01-16,12.46,0.04


In [25]:
display(HTML(opcoes[opcoes['TIPO'] == 'CALL'].dropna().to_html()))

,ATIVO_OBJ,ATIVO,TIPO,MOD,DT_VCTO,STRIKE,PRECO
0,lren3,LRENA1,CALL,A,2026-01-16,16.96,0.01
6,lren3,LRENA119,CALL,E,2026-01-16,11.71,1.70
7,lren3,LRENA121,CALL,A,2026-01-16,11.96,1.32
8,lren3,LRENA124,CALL,E,2026-01-16,12.21,1.08
9,lren3,LRENA126,CALL,A,2026-01-16,12.46,1.15
10,lren3,LRENA129,CALL,E,2026-01-16,12.71,0.73
11,lren3,LRENA133,CALL,A,2026-01-16,12.96,0.47
12,lren3,LRENA135,CALL,E,2026-01-16,13.21,0.31
13,lren3,LRENA138,CALL,A,2026-01-16,13.46,0.20
14,lren3,LRENA140,CALL,E,2026-01-16,13.71,0.12
